(ref:lec7:ex4_sol)=
# Solutions of Exercise 7.4


For the system $G(s) = 5 \frac{s + 7}{(s + 2)^3}$ a colleague of yours has designed the following feedback controller:

$$
F(s) = 10 \frac{s + 2}{s + 7}
$$

The Bode plots of the open-loop transfer function $G_o(s)$ are reported in {numref}`fig:lec7:ex4_sol:bode`,

1. Suggest a range of possible sampling frequencies at which the discretized controller should operate.
2. Pick a value of the sampling frequency, and compute the (algebraic) discretized control law using any method of choice (Forwad Euler, Backward Euler, Trapezoidal rule).
3. What is the loss of phase margin? Is the residual phase margin still sufficient?

In [ ]:
set(groot, 'defaultLineLineWidth', 2);

In [ ]:
s = tf('s');
G =  5 * (s + 7) / (s+2)^3;
F = 10 * (s + 2) / (s+7);
bode(F*G); grid on; box on;

---

## Solution
### Question 1

From {numref}`fig:lec7:ex4_sol:bode`, the crossover frequency is $\omega_c \approx 6.5$ rad/s. In Hz, this is $f_{c} = \frac{\omega_c}{2 \pi} \approx 1$ Hz. According to the rule-of-thumb presented during the lectures, a good value of the sampling frequency $f_s$ is in the range

$$
20 f_c \leq f_s \leq 50 f_c
$$

Therefore, $f_s \in [20, 50]$ Hz. This ensures that the sampling is fast enough for the controller (i.e., it doesn't lead to large losses of phase margins), while not too fast fast (which would lead to unreasonably expensive hardware and to actuator's wear-and-tear).

```{admonition} Common pitfall
:class: critical

A common error is to invoke the Nyquist-Shannon Theorem and answer $f_s > 2 f_c$. This solution is wrong, because the error signal $e(t)$ being sampled is **not** band-limited to $f_c$. The error has indeed frequency components at frequencies higher than $f_c$, therefore this theorem cannot be applied![^shannon]
```

### Question 2

Let's pick $f_s = 25$ Hz, meaning that the sampling time is $T = \frac{1}{25} = 0.04$. We will use the Forward Euler formula ($\alpha = 0$), and replace

$$
s \gets \frac{1}{T} \frac{1 - q^{-1}}{q^{-1}} = 25 \frac{1- q^{-1}}{q^{-1}}
$$

into the continuous-time controller

$$
U(s) = \underbrace{10 \frac{s+2}{s+7}}_{F(s)} E(s)
$$

Apply this substitution we actually translate the control law in the discrete-time domain:

$$
\begin{aligned}
u_k &= 10 \frac{25 \frac{1- q^{-1}}{q^{-1}} + 2}{25 \frac{1- q^{-1}}{q^{-1}} + 7} e_k \\
&= 10 \frac{25 (1- q^{-1}) + 2 q^{-1}}{25 (1- q^{-1}) + 7 q^{-1}} e_k \\
&= 10 \frac{25- 23 q^{-1}}{25 - 18 q^{-1}} e_k 
\end{aligned}
$$


Multiplying both sides by $(25 - 18q^{-1})$, we get

$$
(25 - 18 q^{-1}) u_k = (250 - 230 q^{-1}) e_k
$$

Recalling that $q^{-1}$ is the time-shift operator, we can finally get the algebraic, discrete-time control law by isolating $u_k$:

$$
u_k = \frac{18}{25} u_{k-1} + \frac{250}{25} e_k - \frac{230}{25} e_{k-1} = 0.72 u_{k-1} + 10 e_k - 9.2 e_{k-1}
$$

In practice, this control law can be implemented by, e.g., the following C function invoked every $T=0.04$ seconds:

```{code} C
float control_law(float u_prev, float e_prev, float e_current)
{
    return 0.72f * u_prev
         + 10.0f * e_current
         - 9.2f  * e_prev;
}
```

Curious to see how this kind of controllers can be implemented on Arduino? Check out [this code](https://gist.github.com/careyi3/02a57dfd3a62a96d46171489c83488bd)!

### Question 3

The phase loss due to the discretization procedure is

$$
\Delta \varphi_m \approx \frac{T}{2} \omega_c \frac{180}{\pi} \approx 7.5^\circ
$$

The phase margin achieved by the continuous-time controller was (see {numref}`fig:lec7:ex4_sol:bode`) approximately $\varphi \approx 33$.

The new phase margin is $33^\circ - 7.5^\circ = 25.5^\circ$. While rather low, this phase margin is still enough to guarantee closed-loop stability, at the price of rather large oscillations.

---
[^shannon]: The [Nyquist-Shannon sampling theorem](https://en.wikipedia.org/wiki/Nyquist–Shannon_sampling_theorem) states that if a signal $z(t)$ is band-limited to frequencies $[0, f_{max}]$ (that is, its spectrum is zero at frequencies higher than $f_{max}$), a sampling frequency $f_s > 2 f_{max}$ is sufficient to avoid aliasing, meaning that the signal can be fully reconstructed.